In [14]:
import pandas as pd
import io


dframe=pd.read_csv("C:/Users/PC/Desktop/imp.csv")

In [15]:
#we have to classify the numeric cols and the non numeric  cols
def get_numeric_cols(df):
  return list(df.describe().columns)
def get_categorical_cols(df):
  return list ( set(df.columns) - set(get_numeric_cols(df)) )
numeric_cols=get_numeric_cols(dframe)
categorical_cols=get_categorical_cols(dframe)

#printing both typs pf cols and chechking if we have all the columns in the data frame
print(numeric_cols)
print(categorical_cols)
print(f'len of df:{len(dframe.columns)} = len of numeric cols{len(numeric_cols)} + len categorical cols{len(categorical_cols)} ')


['Longitude', 'Latitude', 'EcoLastRainEndDay', 'EcoLastRainStartDay']
['EcoSoilHumidity', 'Infestation', 'SwarmFlyingFrom', 'EcoVegDensityEst', 'EcoVegetationState', 'SwarmFlyingTo']
len of df:10 = len of numeric cols4 + len categorical cols6 


In [16]:
#lets see the na distribution in the data
dframe.isna().sum()

Longitude                 4
Latitude                  4
SwarmFlyingFrom        8518
SwarmFlyingTo          8523
EcoLastRainEndDay      6321
EcoLastRainStartDay    6321
EcoVegetationState     5330
EcoVegDensityEst       5831
Infestation               0
EcoSoilHumidity        5831
dtype: int64

In [17]:
#filling the na cols using the mean data for the numeric cols
def fill_na_numeric(dframe,numeric_cols):
  df=dframe[numeric_cols].copy()
  for col in df.columns:
    df[col]=df[col].fillna(df[col].mean().astype(int))
  return df

def drop_null_cols(dframe):
  #if there remains a column with na values after possible extrapolation methods , we should completely drop that column
  return dframe.copy().dropna(axis=1)

#assigning the non na cols to a new variable
non_na_df=fill_na_numeric(dframe,numeric_cols)
#droping the complete na cols from the data frame
non_na_df=drop_null_cols(non_na_df)

#normalizing  the values to have the same data structure
def normalize(df):
  df_copy=df.copy()   #copying the dataframe to a new one so we wont lose the previous data
  df_copy= (df_copy.mean() - df_copy ) / df_copy.std()   #using the standard division to normalize the values
  return df_copy.dropna(axis=1)

#normalized_df= normalize(non_na_df)
#mean=normalized_df.mean()
#std=normalized_df.std()
#print(pd.DataFrame({"mean":mean,"std":std}))
numeric_df=non_na_df

In [18]:
#fillinf the na cols in the catagorical cols of our dataframe
def fill_na_categorical(dframe,categorical_cols):
  df=dframe[categorical_cols].copy()
  for col in df.columns:
    df[col]=df[col].fillna(col+"_empty") #giving the na values names which would be better classifier than na
  return df
#geting the na filled cols formour data frame
old_na_counts=dframe[categorical_cols].isna().sum()
print([col for col in old_na_counts.index if old_na_counts[col]>0])
#itrating through the dataframe to assign the values to the na cols
categorical_non_na=fill_na_categorical(dframe,categorical_cols)
new_na_counts=categorical_non_na.isna().sum()
print([col for col in old_na_counts.index if new_na_counts[col]>0])
categorical_df=categorical_non_na

['EcoSoilHumidity', 'SwarmFlyingFrom', 'EcoVegDensityEst', 'EcoVegetationState', 'SwarmFlyingTo']
[]


In [19]:
#mapping the row values of each cols with a specific numbe so the preprocessing is done
categorical_non_na['EcoSoilHumidity'].value_counts()
categorical_mappings={}
def convert_categorical_cols(categorical_df):
  df=categorical_df.copy()
  for col_name in df.columns:
    to_replace=list(set(df[col_name].to_list()))
    categorical_mappings[col_name]=to_replace  #replacing the values with an integer value
    df[col_name].replace(to_replace=to_replace, value=range(1,len(to_replace)+1), inplace=True)
    if(df[col_name].dtype != 'int64'):
      df.drop(col_name,axis=1)
  return df

converted_df=convert_categorical_cols(categorical_df)
categorical_df=converted_df
#list(converted_df.dtypes)
#print(categorical_mappings['SwarmFlyingTo'])

In [20]:
#printing the actual values of the cols and rows and mapping integer values to them
for key in categorical_mappings:
  print(key,categorical_mappings[key])

EcoSoilHumidity ['Dry', 'Slightly moist', 'Wet', 'Moist', 'EcoSoilHumidity_empty']
Infestation ['Not present', 'Present']
SwarmFlyingFrom ['W', 'E', 'SE', 'NW', 'N', 'S', 'SwarmFlyingFrom_empty', 'SW', 'NE']
EcoVegDensityEst ['Low', 'EcoVegDensityEst_empty', 'Medium', 'Dense']
EcoVegetationState ['Green', 'Dry', 'Greening', 'EcoVegetationState_empty', 'Drying']
SwarmFlyingTo ['W', 'E', 'SE', 'NW', 'SwarmFlyingTo_empty', 'N', 'S', 'SW', 'NE']


In [21]:
#concatinating the normalized version of both data types
final_df=pd.concat([numeric_df,categorical_df],axis=1)

In [22]:
#choosing the "swarmFlying To " col as the predictable column and checking corelation to it
final_df.corr()['SwarmFlyingTo'].sort_values(ascending=False).iloc[:15]

SwarmFlyingTo          1.000000
SwarmFlyingFrom        0.185802
EcoLastRainEndDay      0.051609
EcoLastRainStartDay    0.047711
EcoVegetationState     0.007989
Infestation            0.002621
Latitude              -0.004041
Longitude             -0.011434
EcoSoilHumidity       -0.013426
EcoVegDensityEst      -0.019381
Name: SwarmFlyingTo, dtype: float64

In [23]:
#chosing the x and y variables for our prediction
y=final_df['SwarmFlyingTo']
final_df=final_df.drop('SwarmFlyingTo',axis=1)
x=final_df

In [73]:
#chosing a decision tree model for our prediction and training our model
from sklearn.tree import DecisionTreeClassifier
import joblib
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.4)
clf=DecisionTreeClassifier(max_depth=6,min_samples_leaf=3)
#fitting the data on the model
clf.fit(xtrain.values,ytrain.values)
filename =  "DecisionTreeClassifier2.joblib"
joblib.dump(clf, filename)


['DecisionTreeClassifier2.joblib']

In [90]:
#checking he score of how well t trainind the data and the prediction accuracy
print(clf.score(xtrain.values,ytrain.values))
index=ytest!=5
#print(clf.score(xtrain[index].values,ytrain[index].values))
import sklearn
print(clf.get_depth())
print(clf.get_n_leaves())
print(clf.predict_proba(xtest[index].values))
#sklearn.metrics.confusion_matrix(clf.predict(xtest.values),ytest)
#['W', 'E', 'SE', 'NW', 'SwarmFlyingTo_empty', 'N', 'S', 'SW', 'NE']

0.9951681484344801
6
23
[[0.         0.         0.         0.         0.         0.
  1.         0.         0.        ]
 [0.         0.         0.66666667 0.         0.33333333 0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.6
  0.         0.         0.4       ]
 [0.         0.         0.4        0.         0.         0.
  0.         0.6        0.        ]
 [0.         0.         0.         0.         0.96       0.04
  0.         0.         0.        ]
 [0.         0.         0.4        0.         0.         0.
  0.         0.6        0.        ]
 [0.66666667 0.         0.         0.         0.         0.33333333
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         1.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         1.        ]
 [0.         0.         0.         0.         0.         0.
  1.         0.         0.        ]
 [0.6

In [70]:
ytest.value_counts()/ytest.uniqure()

5    3406
1       8
3       8
8       7
2       6
7       5
6       4
9       4
4       2
Name: SwarmFlyingTo, dtype: int64

In [29]:
#testing the raw data inputed from the user (raw based operation)
def predict():
    x=xtest.iloc[213,:]
    y=clf.predict([x])
    #prinitig the result and mapping it back to the column value when inputed
    print(y)
    result = categorical_mappings['SwarmFlyingTo'][y[0]-1]
    print(result)
    return result
predict()

[5]
SwarmFlyingTo_empty


'SwarmFlyingTo_empty'

In [15]:
from sklearn import tree
# import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

def heatmap(clf):
  sns.set(font_scale=1.5)

  sns.heatmap(clf,
            cmap='coolwarm',
            annot=True,
            fmt='.5g',
            vmax=200)
heatmap(clf)

ValueError: Must pass 2-d input. shape=()

In [117]:
clf.feature_names_in_

array(['ID', 'Day', 'Year', 'Latitude degrees', 'Latitude minutes',
       'Latitude seconds', 'Latitude', 'Longitude degrees',
       'Longitude minutes', 'Longitude seconds', 'Longitude',
       'EcoAreaSurveyed', 'EcoLastRainStartDay', 'EcoLastRainEndDay',
       'EcoLastRainYear', 'EcoLastRainAmount', 'EcoDepthDrySoil',
       'EcoDepthWetSoil', 'LocustsAreaInfested', 'BandDensityMin',
       'BandDensityMax', 'BandSizeMin', 'BandSizeMax', 'BandsNo',
       'TransectL', 'TransectW', 'AdDensityComputed', 'SwarmDensityMin',
       'SwarmDensityMax', 'SwarmSizeMin', 'SwarmSizeMax',
       'ControlAppRate', 'ControlQuantity', 'ControlAreaTreated',
       'ControlEst%Kill', 'AdBreedingSol', 'HopAppS', 'HopGregF',
       'SwarmSizeLarge', 'HopTransH', 'EcoSoilHumidity', 'BandH',
       'EcoVegIndicator', 'EcoIndicator', 'HopSolH', 'AdMatSolMature',
       'AdNosEstTrans', 'BandF', 'BandSizeVSmall', 'AdScattered',
       'SwarmBreeding', 'SwarmMatUnknown', 'Observer name',
       'Control

In [124]:
text_representation = tree.export_text(clf,feature_names=clf.feature_names_in_.tolist())
print(text_representation)

|--- SwarmFlyingFrom <= 4.50
|   |--- SwarmFlyingFrom <= 3.50
|   |   |--- SwarmFlyingFrom <= 1.50
|   |   |   |--- SwarmSizeMin <= 12.50
|   |   |   |   |--- class: 2
|   |   |   |--- SwarmSizeMin >  12.50
|   |   |   |   |--- class: 7
|   |   |--- SwarmFlyingFrom >  1.50
|   |   |   |--- Latitude minutes <= 8.00
|   |   |   |   |--- class: 1
|   |   |   |--- Latitude minutes >  8.00
|   |   |   |   |--- Day <= 26.00
|   |   |   |   |   |--- Latitude seconds <= 29.00
|   |   |   |   |   |   |--- Latitude <= 11.65
|   |   |   |   |   |   |   |--- class: 7
|   |   |   |   |   |   |--- Latitude >  11.65
|   |   |   |   |   |   |   |--- class: 2
|   |   |   |   |   |--- Latitude seconds >  29.00
|   |   |   |   |   |   |--- SwarmDensityLow <= 1.50
|   |   |   |   |   |   |   |--- class: 3
|   |   |   |   |   |   |--- SwarmDensityLow >  1.50
|   |   |   |   |   |   |   |--- class: 8
|   |   |   |   |--- Day >  26.00
|   |   |   |   |   |--- class: 5
|   |--- SwarmFlyingFrom >  3.50
|   |  